In [1]:
from researcher import Researcher
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [2]:
    
WEBHOOK_HOST = "0.0.0.0"
WEBHOOK_PORT = 8042
WEBHOOK_BASE = ""
ADMIN_URL = "http://researcher-agent:8041"
# ADMIN_URL = "http://0.0.0.0:8041"

agent_config = {
    "webhook_host": WEBHOOK_HOST,
    "webhook_port": WEBHOOK_PORT,
    "webhook_base": WEBHOOK_BASE,
    "admin_url": ADMIN_URL
}



In [3]:
import torch

# models
from torch import nn
from torch import optim
from torch.autograd import Variable
import os
import sys

from opacus import PrivacyEngine

# The Researcher generates the model

model = nn.Sequential(
            nn.Linear(8, 4),
            nn.Sigmoid(),
            nn.Linear(4, 2),
            nn.Sigmoid(),
            nn.Linear(2, 1),
            nn.Sigmoid()
)

In [4]:
validation_data_path = 'data/coordinator_validation_data.csv'

In [5]:
researcher = Researcher(agent_config, validation_data_path, model)

Subscribing too: basicmessages
Subscribing too: connections
Subscribing too: issue_credential
Subscribing too: present_proof
CLEANING THE VALIDATION SET
VALIDATION SET HAS BEEN CLEANED


## Request Authorised Research Credential

Copy invite from the [Health Research Authority Notebook](http://localhost:8889/lab/tree/Health%20Research%20Regulator.ipynb)

In [6]:
# invite = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '6cccd38d-b34b-4798-aa7c-b529ddd315e0', 'recipientKeys': ['Fm7bvrNNTJCuT1RKjPqcNg96Xcamm6mwivNgQ7CpU8ph'], 'serviceEndpoint': 'http://172.17.0.1:8030', 'label': 'NHS Health Research Authority'}
# invite = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': '8cceb9cb-e2f1-408b-b4ac-6edf72fc6308', 'recipientKeys': ['5ssrrfvjGvaTUV1RGhChdjCwFxYHckVZiEvSWP7Tcpfw'], 'label': 'NHS Health Research Authority', 'serviceEndpoint': 'http://172.17.0.1:8030'}

# invite = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'd9a96a35-5d7a-4980-b6b4-cd7cbecbbb3a', 'label': 'NHS Health Research Authority', 'serviceEndpoint': 'http://172.17.0.1:8030', 'recipientKeys': ['6Ug1bMo6UatGWBGvAQwLVbJmfjJUabFsRgNKrr7yYex5']}

invite = {'@type': 'did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation', '@id': 'cb3915e4-a2e5-4286-96c6-9887cfecdfda', 'recipientKeys': ['5Fmf4c8NMZ39fvv1QSJQDxXsNMCQ7r6fS5EBUacuxHAv'], 'label': 'NHS Lothian', 'serviceEndpoint': 'http://172.17.0.1:8020'}

In [7]:
await researcher.agent_controller.connections.accept_connection(invite)

Connection Handler Called
Connection 37d6eee2-6446-4108-a1ce-d4882df9ee12 in State invitation
Connection Handler Called
Connection 37d6eee2-6446-4108-a1ce-d4882df9ee12 in State request


{'created_at': '2021-07-24 16:41:17.304453Z',
 'invitation_key': '5Fmf4c8NMZ39fvv1QSJQDxXsNMCQ7r6fS5EBUacuxHAv',
 'request_id': '69853d18-f22a-4831-9593-c2f7be30f3ea',
 'routing_state': 'none',
 'invitation_mode': 'once',
 'their_role': 'inviter',
 'rfc23_state': 'request-sent',
 'my_did': '9H44a1CmZ1a1Crih14PcRW',
 'accept': 'manual',
 'state': 'request',
 'connection_id': '37d6eee2-6446-4108-a1ce-d4882df9ee12',
 'their_label': 'NHS Lothian',
 'updated_at': '2021-07-24 16:41:17.705786Z'}

## Configure Authentication Policy

Note: Normally we would specify the credential definition here specifying which entities we trust to issue this schema, but this changes every time because it's a demo.

Feel free to copy yours across from the NHS Trust though.

In [8]:


schema_id = "BvRKFu1fdzGdJzQcKb8e8p:2:NHS Hosptial:0.0.1"
# cred_def = "WfntKNFwXMQfgmU9ofbxPM:3:CL:156569:default"

revocation = False
exchange_tracing = False

#Enable this to ask for attributes to identity a user
#TODO - change restriction to schemaId or credentialId

req_attrs = [
    {"name": "name", "restrictions": [{"schema_id": schema_id}]},#, "cred_def_id": cred_def}]},
]

global indy_proof_request
indy_proof_request = {
    "name": "Proof of Hospital",
    "version": "1.0",
    "requested_attributes": {
        f"0_{req_attr['name']}_uuid":
        req_attr for req_attr in req_attrs
    },
    "requested_predicates": {
#         f"0_{req_pred['name']}_GE_uuid":
#         req_pred for req_pred in req_preds
    },
}



In [9]:
researcher.set_authentication_policy(indy_proof_request)

## Send Invite to Hospital 1

Option: We could make this a multiuse invite.

In [ ]:
researcher.new_dataowner_invite()

Connection Handler Called
Connection d8fc8918-a414-4d09-83f0-3db07bb64573 in State invitation

♫♫♫ > STEP 1: Send the aries invitation to the data owner

{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation", "@id": "3cb940ad-7294-4bae-a001-284b1b0ab842", "serviceEndpoint": "http://172.17.0.1:8040", "label": "Health Researcher", "recipientKeys": ["89HLw1FSm1Fy547oy1tEi6DryzRDoM6AFjNmzKBFyDEf"]}

Connection Handler Called
Connection d8fc8918-a414-4d09-83f0-3db07bb64573 in State request
Accepting request  d8fc8918-a414-4d09-83f0-3db07bb64573
Connection Handler Called
Connection d8fc8918-a414-4d09-83f0-3db07bb64573 in State response
Sending trust ping d8fc8918-a414-4d09-83f0-3db07bb64573
Connection Handler Called
Connection d8fc8918-a414-4d09-83f0-3db07bb64573 in State active
Pending dataowner connection moved to active. Challenging with auth policy if set
Handle present proof
Role verifier, Exchange 23759f9d-b21c-4098-baa8-5a3d08ce5726 in state request_sent


## Send Invite to Hospital 2

In [ ]:
researcher.new_dataowner_invite()

Connection Handler Called
Connection 9ce84d69-1c4e-4e7c-89b4-4bb1058ae5f7 in State invitation

♫♫♫ > STEP 1: Send the aries invitation to the data owner

{"@type": "did:sov:BzCbsNYhMrjHiqZDTUASHg;spec/connections/1.0/invitation", "@id": "7726f1fe-b674-4729-ab80-cdc056df103f", "recipientKeys": ["8ARx1iCpCtmLDib8UMYGk8s3RbwGMHmPhLiBgdU8aSWL"], "serviceEndpoint": "http://172.17.0.1:8040", "label": "Health Researcher"}

Connection Handler Called
Connection 9ce84d69-1c4e-4e7c-89b4-4bb1058ae5f7 in State request
Accepting request  9ce84d69-1c4e-4e7c-89b4-4bb1058ae5f7
Connection Handler Called
Connection 9ce84d69-1c4e-4e7c-89b4-4bb1058ae5f7 in State response
Sending trust ping 9ce84d69-1c4e-4e7c-89b4-4bb1058ae5f7


Connection Handler Called
Connection 9ce84d69-1c4e-4e7c-89b4-4bb1058ae5f7 in State active
Pending dataowner connection moved to active. Challenging with auth policy if set
Handle present proof
Role verifier, Exchange 5a880bf6-847a-4f37-97f3-afd684d9e660 in state request_sent
Handle present proof
Role prover, Exchange 054c34e1-a25a-4632-94f6-d325c8709814 in state request_received
Received Authentication Challenge
Credentials []


## Send Invite to Hospital 3

In [ ]:
researcher.new_dataowner_invite()

## Initiate Learning

PPML Learning messages are sent over DIDComm using the basic message protocol. You will be able to see the encrypted message in the terminal you initialised this project.

In [11]:
researcher.initiate_learning()

MODEL OPENED FOR TRANSPORT
Training model with 1 DataOwners
HOSPITAL MODEL LOADED

PRINTING PARAMETERS:


Sequential(
  (0): Linear(in_features=8, out_features=4, bias=True)
  (1): Sigmoid()
  (2): Linear(in_features=4, out_features=2, bias=True)
  (3): Sigmoid()
  (4): Linear(in_features=2, out_features=1, bias=True)
  (5): Sigmoid()
)


0.weight tensor([[-0.2710, -0.9557,  0.2309, -1.9110, -0.9440, -0.4835,  1.8869,  3.2344],
        [-0.7837, -0.9277,  1.5270, -2.2951, -0.0703,  2.0951, -0.2947,  0.4189],
        [ 0.3087,  0.1126, -0.6512,  0.4391, -0.7844, -0.3249, -0.5029,  3.2722],
        [-2.2386,  1.0693, -1.0953, -1.6919, -0.6925,  0.4706,  0.1246,  0.6962]])
0.bias tensor([-1.0267, -0.2995, -2.2666,  1.8172])
2.weight tensor([[ 1.7648,  1.0091,  1.9958, -2.0196],
        [-2.6523, -2.2176, -2.9162,  4.3020]])
2.bias tensor([-1.2612,  2.2130])
4.weight tensor([[ 4.0461, -6.9314]])
4.bias tensor([0.5980])



HOSPITAL IS VALIDATING
Model loss on validation set:  tensor(-0.7815